<a href="https://colab.research.google.com/github/githubforkj/signate/blob/main/mufj/02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# フォーラムで提示された強プレイヤーの写経

(完全理解も同時に行う)

## 準備

In [ ]:
# signateとの連携

!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 49.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=dd58651f6fb9e468153e448666cc345581c13876392f6082a57829d9ffce6200
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
requests 2.23.0 requires urllib3!

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 実装

In [ ]:
%%capture
!pip install transformers datasets sentencepiece

In [ ]:
!nvidia-smi

Mon Sep 12 01:06:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Import

In [ ]:
from pathlib import Path
from multiprocessing import cpu_count
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, f1_score

from datasets import load_dataset, Dataset, DatasetDict

from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction
from transformers import set_seed

import torch
from torch import nn

### Config

In [ ]:
# 実験番号
EXP_NAME = 'exp02'
# ディレクトリ
INPUT_DIR = Path('/content/drive/MyDrive/signate/mufj/Input')
OUTPUT_DIR = Path('/content/drive/MyDrive/signate/mufj/Output')
# デバッグ用
DEBUG = True
TEXT_COLUMNS = ['goal','country','duration','category1','category2','html_content']
MODEL_NAME = 'microsoft/deberta-v3-base'
N_SPLIT = 5
TRN_FOLDS = [0,1,2,3,4]
MAX_LEN = 512
SEED = 3090


training_args = TrainingArguments(
    output_dir = str(OUTPUT_DIR/EXP_NAME),
    seed = SEED,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs = 4,
    weight_decay = 0.01,
    fp16 = True,
    gradient_accumulation_steps=1,
    evaluation_strategy='epoch',
    save_total_limit = 1,
    save_strategy = 'epoch',
    metric_for_best_model = 'f1_score',
    load_best_model_at_end = True,
    greater_is_better=True
)

### Cross Validation ⌘

In [ ]:
def create_folds(data, num_splits):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits = num_splits, shuffle=True, random_state=42)
    data_labels = data['state']

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        # f:enumerateで振られた番号
        # (t_, v_):二つ目はタプルになっているため、これで通る。
        # (num,(np.array,np.array))
        data.loc[v_, "kfold"] = f
        # v_(2分割した9870個のデータの片方。)の番号でインデックス参照し、"kfold"列の値を0,1,2,3,4(5分割の場合)のどれかに置き換える。
        
    
    return data


data = pd.read_csv(INPUT_DIR / "train.csv")
data = create_folds(data, num_splits=N_SPLIT)
data.to_csv("train_folds.csv", index = False) #セッションでのみ作成される
print("Folds created sucessfully")

Folds created sucessfully


### Preprocess ⌘

In [ ]:
# テキストのトークン化&トークンをinput_idに変更
def text_to_input_ids(examples):
    return tokenizer(examples['text'] ,padding=False ,truncation=True, max_length=MAX_LEN)
    # padding:文字数を揃える
    # truncation:特定以下の文字数のトークンを切り捨て

def connect_text(df, text_cols, sep):
    text_df = df[text_cols].fillna('NAN').astype(str)
    # text_colsの空白をNANで埋める
    connected_text = text_df[text_cols[0]].str.cat(text_df[text_cols[1:]], sep=sep)
    # pandasのcat関数を使用して、各列の文字列を横に結合する(連結文字を[SEP]とする。)

    return connected_text

### Augumentation

expanding data

In [ ]:
class RandomMask:
    def __init__(self, tokenizer, prob=0.5, mask_prob=0.15):
        self.mask_token_id = tokenizer.mask_token_id
        self.prob = prob
        self.mask_prob = mask_prob
    
    def __call__(self, examples):
        if random.random() > self.prob:
            # マスキングを行う
            examples['input_ids'] = [self._mask(input_ids) for input_ids in examples['input_ids']]

        return examples
    
    def _mask(self, input_ids: list) -> list:
        length = len(input_ids)
        # ランダムに15%選ぶ
        mask_idx = random.sample(range(1, length), int(length*self.mask_prob)) # random masking except [CLS]
        # mask_idxに入っているidに該当するidのtoken_idをmask_token_idに置き換える
        for i in mask_idx:
            input_ids[i] = self.mask_token_id

        return input_ids

### Trainer

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels').float()
        _inputs = {k: v for k, v in inputs.items() if k != 'labels'} #to not automatically calculate losses

        outputs = model(**_inputs)
        logits = outputs.get('logits')

        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(logits.view(-1), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

# 第一引数に予測したラベル、第二引数にその確率
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        # sklearn
        score = f1_score(y_true = y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 学習結果を出力するための関数？(p: EvalPredictionと引数pにアノテーションをつけている。)
def compute_metrics(p: EvalPrediction):
    # シグモイド関数を呼び出し、
    preds = sigmoid(p.predictions)
    labels = p.label_ids
    score = threshold_search(labels, preds)['f1']
    metrics = {'f1_score': score}

    return metrics
        

### Main

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv('train_folds.csv')
    test_df = pd.read_csv(INPUT_DIR / 'test.csv')
    # if DEBUG:
    #     df = df.head(50)
    #     test_df = test_df.head(50)
    #     training_args.num_train_epochs = 1

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # AutoTokenizerは様々なtokenizerのラッパー。MODEL_NAMEに入れるモデルによって中身が変わる。
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #for dynamic padding
    # データを集めてミニバッチを作成する。withpaddingを指定することで空いている箇所には0が埋まる
    # transformerのモジュール。

    # 上で定義したRandomMaskクラスを使用する。tokenの15%をマスキングする事前学習の設定。
    train_transform = RandomMask(tokenizer, prob=0.5, mask_prob=0.1) #[MASK] augmentation


    '''
        前処理
    '''
    df['text'] = connect_text(df, TEXT_COLUMNS, tokenizer.sep_token)
    # dfの最右列に'text'列を作成する。
    df['label'] = df['state']
    # dfの最右列に'state'列を作成する
    ds = Dataset.from_pandas(df[['text','label','kfold']])
    # pandasのdataframeをdatasetに。
    ds = ds.map(text_to_input_ids, batched=True, num_proc=cpu_count())
    # map関数は、シーケンスの構成要素全てに対して、ある関数の処理を行わせる
    # input_ids,token_type_ids,attention_mask列が追加される

    # test_dfに関しても上のdfと同様に前処理を行う
    test_df['text'] = connect_text(test_df, TEXT_COLUMNS, tokenizer.sep_token)
    test_ds = Dataset.from_pandas(test_df[['text']])
    test_ds = test_ds.map(text_to_input_ids, batched=True, num_proc=cpu_count())
    # なぜ'text'カラムを削除する？？か分からない
    test_ds = test_ds.remove_columns(['text'])

    '''
        前処理ここまで
    '''


    oof = []
    labels = []
    test_preds = []
    for fold in TRN_FOLDS:
        set_seed(SEED)
        # dictionary形式で持つ
        ds_i = DatasetDict(
            # foldは0,1,2,3,4のどれか
            {
                # 学習用のデータ7832個をfilter関数を用いて割り当てる。
                'train': ds.filter(lambda x: x['kfold']!=fold),
                # 評価用のデータ1959個を同上
                'eval': ds.filter(lambda x: x['kfold']==fold),
            }
        )
        # ds_iから'text','kfold'を除外する
        ds_i = ds_i.remove_columns(['text','kfold'])
        # ds_i['train'].set_transform(train_transform)

        '''
            学習
        '''

        # モデルのインスタンス化
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)
        # OUTPUTディレクトリに出力する
        training_args.output_dir = str((OUTPUT_DIR / EXP_NAME) / f'fold_{fold}')
        # 学習パラメータ設定
        trainer = CustomTrainer(

            model = model,
            # 出力先
            args = training_args,
            # 学習用データセット
            train_dataset = ds_i["train"],
            # 検証用データセット(評価しなければなくても良い)
            eval_dataset=ds_i["eval"],
            tokenizer = tokenizer,
            data_collator=data_collator,
            # 上で定義した関数。デフォルトだと結果が表示されない。
            compute_metrics=compute_metrics,
        )

        # 学習！
        trainer.train()

        # inference
        # 検証用データセットの推論(trainer.evaluate()でも良いのでは？)
        oof_i = trainer.predict(ds_i['eval'])
        # テスト用データセットでの推論
        test_preds_i = trainer.predict(test_ds).predictions
        # EvalPredictionクラスはpredictionsとlabel_idsが定義されている。
        # 活性化関数sigmoidを通す
        oof.append(sigmoid(oof_i.predictions))
        # label_idsをlabelsリストに格納
        labels.append(oof_i.label_ids)
        # テスト用データセットの推論結果を格納
        test_preds.append(test_preds_i)
    
    # ？？？
    oof = np.vstack(oof).flatten()
    # ？？？
    labels = np.hstack(labels)
    # 上で定義した最適な閾値を探す関数をインスタンス化
    search_result = threshold_search(labels, oof)
    print('OOF Score: ', search_result['f1'], 'Threshold: ', search_result['threshold'])

    # 予測値の平均が閾値よりも高いものを採用している？？
    test_df['label'] = (np.mean(test_preds, axis=0) > search_result['threshold']).astype(int)
    # 結果を出力
    test_df[['id','label']].to_csv((OUTPUT_DIR/EXP_NAME)/ 'sub.csv', header=False, index=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

...

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

Epoch,Training Loss,Validation Loss,F1 Score
1,No log,0.508516,0.785952


***** Running Evaluation *****
  Num examples = 1959
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/signate/mufj/Output/exp01/fold_0/checkpoint-490
Configuration saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_0/checkpoint-490/config.json
Model weights saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_0/checkpoint-490/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_0/checkpoint-490/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_0/checkpoint-490/special_tokens_map.json


Epoch,Training Loss,Validation Loss,F1 Score
1,No log,0.508516,0.785952


KeyboardInterrupt: ignored